In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
from scipy.stats import pearsonr
import seaborn as sns

In [2]:
def convert_data2string(df):
    vals = df.values.astype(str)
    vals = np.array([' '.join(x) for x in vals])

    # should be fine now, but this checks that nothing is getting truncated due to type casting
    test = np.array([[y.isdigit() for y in x.split(' ')] for x in vals])
    assert (test.sum(axis=-1) == df.shape[-1]).all()

    return vals

In [3]:
# Blocks random
files_orig_rand = glob.glob('../results/init_random/blocks/*original*.t7')
files_rec_rand = glob.glob('../results/init_random/blocks/*reconstructed*.t7')
files_orig_rand = sorted(files_orig_rand)
files_rec_rand = sorted(files_rec_rand)
print(len(files_orig_rand), len(files_rec_rand))

299 299


In [ ]:
# The total number of records, and the total number of record prototypes in the dataset
records_in_orig = 0
prototypes_in_orig = 0
for i in range(len(files_orig_rand)):
    orig_data = torch.load(files_orig_rand[i])
    uniq_orig_data = orig_data.drop_duplicates().reset_index(drop=True)
    records_in_orig+= len(orig_data)
    prototypes_in_orig += len(uniq_orig_data)
print('Blocks')
print('Records in D:', records_in_orig)
print('Prototypes in D:', prototypes_in_orig)

In [ ]:
# The total number of records, and the total number of record prototypes in the reconstructions (random)
records_in_rec = 0
prototypes_in_rec = 0
for i in range(len(files_rec_rand)):
    rec_data = torch.load(files_rec_rand[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])
    
    uniq_rec_data = all_rec_data.drop_duplicates().reset_index(drop=True)
    records_in_rec+= len(all_rec_data)
    prototypes_in_rec+= len(uniq_rec_data)

print('Blocks (random)')
print('Records in D\':', records_in_rec)
print('Prototypes in D\':', prototypes_in_rec)

In [4]:
# The total number of records, and the total number of record prototypes in the reconstructions that are not in the dataset (random)
in_orig_rand = []
in_rec_rand = []

for i in range(len(files_orig_rand)):
    orig_data = torch.load(files_orig_rand[i])
    rec_data = torch.load(files_rec_rand[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    uniq_rec_data = all_rec_data.drop_duplicates().reset_index(drop=True)
    uniq_rec_data_str = convert_data2string(uniq_rec_data)
    
    
    for prototype in uniq_rec_data_str:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_rand.append(n_in_orig)
        in_rec_rand.append(n_in_rec)
            
in_orig_rand = np.array(in_orig_rand)
in_rec_rand = np.array(in_rec_rand)
idxs = (in_orig_rand == 0)
print('Block (random):')
print('Records in D\':', in_rec_rand[idxs].sum()) 
print('Prototypes in D\':', len(in_rec_rand[idxs]))

Block (random):
Records in D': 3696675
Prototypes in D': 161346


In [8]:
# Percentage of appearances in reconstructions of rare record prototypes that occur once, twice, or three times in D
idxs1 = (in_orig_rand == 1) * (in_rec_rand == 1)
idxs2 = (in_rec_rand == 1)
r1 = in_rec_rand[idxs1].sum()/in_rec_rand[idxs2].sum()

print('Block (random) 1 in D:{:0.2f}%'.format(r1*100))

idxs1 = (in_orig_rand == 2) * (in_rec_rand <= 2)
idxs2 = (in_rec_rand <= 2)
r2 = in_rec_rand[idxs1].sum()/in_rec_rand[idxs2].sum()

print('Block (random) 2 in D:{:0.2f}%'.format(r2*100))


idxs1 = (in_orig_rand == 3) * (in_rec_rand <= 3)
idxs2 = (in_rec_rand <= 3)
r3 = in_rec_rand[idxs1].sum()/in_rec_rand[idxs2].sum()

print('Block (random) 3 in D:{:0.2f}%'.format(r3*100))

Block (random) 1 in D:0.20%
Block (random) 2 in D:0.15%
Block (random) 3 in D:0.12%


In [4]:
# The total number of records, and the total number of record prototypes in the dataset that are not in the reconstructions(random)

in_orig_rand = []
in_rec_rand = []

for i in range(len(files_orig_rand)):
    orig_data = torch.load(files_orig_rand[i])
    rec_data = torch.load(files_rec_rand[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    uniq_orig_data_str = convert_data2string(orig_data.drop_duplicates().reset_index(drop=True))
    
    for prototype in uniq_orig_data_str:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_rand.append(n_in_orig)
        in_rec_rand.append(n_in_rec)
            
in_orig_rand = np.array(in_orig_rand)
in_rec_rand = np.array(in_rec_rand)
idxs = (in_rec_rand == 0)
print('Block (random):')
print('Records in D:', in_orig_rand[idxs].sum()) 
print('Prototypes in D:', len(in_orig_rand[idxs]))

Block (random):
Records in D: 1142
Prototypes in D: 651


In [ ]:
# For visualizing correlation
in_orig_rand = []
in_rec_rand = []

for i in range(len(files_orig_rand)):
    orig_data = torch.load(files_orig_rand[i])
    rec_data = torch.load(files_rec_rand[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_prototypes = orig_data.drop_duplicates().reset_index(drop=True)
    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    orig_data_prototypes = convert_data2string(orig_data_prototypes)
    
    for prototype in orig_data_prototypes:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_rand.append(n_in_orig)
        in_rec_rand.append(n_in_rec)
            

in_orig_rand = np.array(in_orig_rand)
in_rec_rand = np.array(in_rec_rand)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi= 300, facecolor='w', edgecolor='k')

ax = sns.scatterplot(x = in_orig_rand, y = in_rec_rand,
                edgecolor = "black", hue = in_rec_rand)
ax.set(xlabel='Occurences in D', ylabel='Frequency in reconstructions')
pr = pearsonr(in_orig_rand, in_rec_rand)
# legend.get_frame().set_edgecolor('1.0')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(0.5)
ax.spines['left'].set_linewidth(0.5)

# # set the x-spine (see below for more info on `set_position`)
ax.spines['left'].set_position('zero')
# turn off the right spine/ticks
ax.spines['right'].set_color('none')
ax.yaxis.tick_left()
# set the y-spine
ax.spines['bottom'].set_position('zero')
# turn off the top spine/ticks
ax.spines['top'].set_color('none')
ax.xaxis.tick_bottom()
ax.get_legend().remove()

plt.xlabel('Occurences in D')
plt.ylabel('Frequency in reconstructions')
plt.figtext(0.2, 0.65, f'Pearson\'s r  = {pr.statistic:0.4f}')
plt.tight_layout()
plt.savefig('figures/corr_block_random.png', dpi = 300)
plt.show()


In [25]:
# Blocks baseline
files_orig_base = glob.glob('../results/init_baseline/blocks/*original*.t7')
files_rec_base = glob.glob('../results/init_baseline/blocks/*reconstructed*.t7')
files_orig_base = sorted(files_orig_base)
files_rec_base = sorted(files_rec_base)
print(len(files_orig_base), len(files_rec_base))

299 299


In [ ]:
# The total number of records, and the total number of record prototypes in the reconstructions (baseline)
records_in_rec = 0
prototypes_in_rec = 0
for i in range(len(files_rec_base)):
    rec_data = torch.load(files_rec_base[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])
    
    uniq_rec_data = all_rec_data.drop_duplicates().reset_index(drop=True)
    records_in_rec+= len(all_rec_data)
    prototypes_in_rec+= len(uniq_rec_data)

print('Block (baseline)')
print('Records in D\':', records_in_rec)
print('Prototypes in D\':', prototypes_in_rec)

In [6]:
# The total number of records, and the total number of record prototypes in the reconstructions 
#that are not in the dataset (baseline)
in_orig_base = []
in_rec_base = []

for i in range(len(files_orig_base)):
    orig_data = torch.load(files_orig_base[i])
    rec_data = torch.load(files_rec_base[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    uniq_rec_data = all_rec_data.drop_duplicates().reset_index(drop=True)
    uniq_rec_data_str = convert_data2string(uniq_rec_data)
    
    
    for prototype in uniq_rec_data_str:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_base.append(n_in_orig)
        in_rec_base.append(n_in_rec)
            
in_orig_base = np.array(in_orig_base)
in_rec_base = np.array(in_rec_base)
idxs = (in_orig_base == 0)
print('Block (baseline):')
print('Records in D\':', in_rec_base[idxs].sum()) 
print('Prototypes in D\':', len(in_rec_base[idxs]))

Block (baseline):
Records in D': 980412
Prototypes in D': 88507


In [7]:
# Percentage of appearances in reconstructions of rare record prototypes that occur once, twice, or three times in D
idxs1 = (in_orig_base == 1) * (in_rec_base == 1)
idxs2 = (in_rec_base == 1)
r1 = in_rec_base[idxs1].sum()/in_rec_base[idxs2].sum()

print('Block (baseline) 1 in D:{:0.2f}%'.format(r1*100))

idxs1 = (in_orig_base == 2) * (in_rec_base <= 2)
idxs2 = (in_rec_base <= 2)
r2 = in_rec_base[idxs1].sum()/in_rec_base[idxs2].sum()

print('Block (baseline) 2 in D:{:0.2f}%'.format(r2*100))


idxs1 = (in_orig_base == 3) * (in_rec_base <= 3)
idxs2 = (in_rec_base <= 3)
r3 = in_rec_base[idxs1].sum()/in_rec_base[idxs2].sum()

print('Block (baseline) 3 in D:{:0.2f}%'.format(r3*100))

Block (baseline) 1 in D:0.15%
Block (baseline) 2 in D:0.06%
Block (baseline) 3 in D:0.03%


In [7]:
# The total number of records, and the total number of record prototypes in the dataset that are not in the reconstructions(baseline)

in_orig_base = []
in_rec_base = []

for i in range(len(files_orig_base)):
    orig_data = torch.load(files_orig_base[i])
    rec_data = torch.load(files_rec_base[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    uniq_orig_data_str = convert_data2string(orig_data.drop_duplicates().reset_index(drop=True))
    
    for prototype in uniq_orig_data_str:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_base.append(n_in_orig)
        in_rec_base.append(n_in_rec)
            
in_orig_base = np.array(in_orig_base)
in_rec_base = np.array(in_rec_base)
idxs = (in_rec_base == 0)
print('Block (baseline):')
print('Records in D:', in_orig_base[idxs].sum()) 
print('Prototypes in D:', len(in_orig_base[idxs]))

Tract (baseline):
Records in D: 293
Prototypes in D: 199


In [ ]:
# For visualizing correlation

in_orig_base = []
in_rec_base = []

for i in range(len(files_orig_base)):
    orig_data = torch.load(files_orig_base[i])
    rec_data = torch.load(files_rec_base[i])
    all_rec_data = None
    for d in rec_data:
        if all_rec_data is None:
            all_rec_data = d
        else:
            all_rec_data = pd.concat([all_rec_data, d])

    orig_data_prototypes = orig_data.drop_duplicates().reset_index(drop=True)
    orig_data_str = convert_data2string(orig_data)
    all_rec_data_str = convert_data2string(all_rec_data)
    orig_data_prototypes = convert_data2string(orig_data_prototypes)
    
    for prototype in orig_data_prototypes:
        n_in_orig = len(np.where(orig_data_str == prototype)[0])
        n_in_rec = len(np.where(all_rec_data_str == prototype)[0])
        in_orig_base.append(n_in_orig)
        in_rec_base.append(n_in_rec)
            

in_orig_base = np.array(in_orig_base)
in_rec_base = np.array(in_rec_base)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi= 300, facecolor='w', edgecolor='k')

ax = sns.scatterplot(x = in_orig_base, y = in_rec_base,
                edgecolor = "black", hue = in_rec_base)
ax.set(xlabel='Occurences in D', ylabel='Frequency in reconstructions')
pr = pearsonr(in_orig_base, in_rec_base)
# legend.get_frame().set_edgecolor('1.0')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(0.5)
ax.spines['left'].set_linewidth(0.5)

# # set the x-spine (see below for more info on `set_position`)
ax.spines['left'].set_position('zero')
# turn off the right spine/ticks
ax.spines['right'].set_color('none')
ax.yaxis.tick_left()
# set the y-spine
ax.spines['bottom'].set_position('zero')
# turn off the top spine/ticks
ax.spines['top'].set_color('none')
ax.xaxis.tick_bottom()
ax.get_legend().remove()

plt.xlabel('Occurences in D')
plt.ylabel('Frequency in reconstructions')
plt.figtext(0.2, 0.65, f'Pearson\'s r  = {pr.statistic:0.4f}')
plt.tight_layout()
plt.savefig('figures/corr_block_baseline.png', dpi = 300)
plt.show()